This review tries to examine the effects of factors including playing surfaces on player movements and by extension, their concurrence with lower extremity injuries.  

The analysis uncovers specific features and feature interactions which combined with player movement present ideal conditions for risk of injury. 

Our analysis discovers that the specific set of factors that influence lower extremity injuries are mutually exclusive to types of injuries. 

Our analysis also reinforces the idea that synthetic playing surfaces play a major role in occurrence of these injuries. 

The analysis is bi-directional: where features are found to promote risk, we look for ways to reduce or eliminate those factors. Where features are found to have low correlation to risk, we look for ways to magnify or encourage their occurrence. 

INTRODUCTION 

Our analysis examined eight crowd-sourced hypotheses: 

Special teams: That players on special teams were more prone to injuries 

Injuries during practice/preseason: That players were prone to more injuries during practice as there are more practice sessions that regular season games 

Backups more prone to injury: That substitutes were more prone to injuries 

Coaching decisions: That coaching decisions have an influence on injuries 

Coming back from injury/% healthy: That players were more prone to injury immediately after coming back from injury 

Injured in cold temperatures/more start/stops: That ‘slower’ games have a higher prevalence of injury 

Same injury, more severe recovery time on synthetic: That synthetic turf not only had a higher prevalence of injuries but a higher prevalence of more severe injuries 

Less injuries during rain/snow: That the effect of natural versus synthetic turf is eliminated during games with rain or snow. 

Games with multiple injuries: That there are games with perfect conditions to cause multiple injuries. 

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#### WE ANALYZE INJURIES BY GAMEID AND NOT PLAYKEY AS 28 INJURIES ARE MISSING PLAYKEYS
##################################################################################################################################
#'Knee', 'Foot', 'Ankle', 'Toes', 'Heel'

###########MAKE CHOICE OF INJURY###### DONT FORGET BRACKETS
choice='Knee'

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc

In [ ]:
players=pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayerTrackData.csv')
orig_cols=players.columns
gc.collect()

Checking that playkeys are unique

In [ ]:
idx=players.groupby(['PlayKey'])['time'].transform(max) == players['time'] 
players[idx]['PlayKey'].value_counts()

In [ ]:
len(players[idx]['PlayKey'])

In [ ]:
#Handling missing data because of selection bias in trees(PlayKey      28)


Characteristics of the dataset included InjuryRecord missing 28 PlayKeys, PlayerTrackData missing 74,526,875 event values and PlayList missing 16910 StadiumType, 18691 Weather and 367 PlayType values. This is consequential because tree models have a selection bias against columns with missing values and imputed values might skew the analysis. We note the columns as they are likely not to depended on for analysis. 

In [ ]:
#Handling missing data because of selection bias in trees(event 74526875)
players.isna().sum()

In [ ]:
#Handling missing data because of selection bias in trees(StadiumType 0.063332, Weather 0.070002,PlayType 0.001375)


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [ ]:
%time players=reduce_mem_usage(players)

In [ ]:
def mapweatherandstadium(df):
  df['IsWet']=df['Weather'].astype(str).str.lower().apply(lambda x: 1 if 'rain' in x or 'rainy' in x or 'snow' in x  or 'thunder' in x else 0)
  df['IsSunny']=df['Weather'].astype(str).str.lower().apply(lambda x: 1 if 'sunny' in x or 'sun' in x or 'clear' in x else 0)
  df['IsSnow']=df['Weather'].astype(str).str.lower().apply(lambda x: 1 if 'snow' in x else 0)
  df['IsCloudy']=df['Weather'].astype(str).str.lower().apply(lambda x: 1 if 'cloudy' in x or 'cloud' in x or 'coudy' in x or 'overcast' in x or 'hazy' in x or 'clouidy' in x else 0)
  df['IsControlled']=df['Weather'].astype(str).str.lower().apply(lambda x: 1 if 'climate controlled' in x or 'Indoor' in x or 'controlled climate' in x else 0)
  #df['IsWeatherUnknown']=df['Weather'].astype(str).str.lower().apply(lambda x: 1 if 'unknown' in x else 0)
  
  df['IsStadiumUnknown']=df['StadiumType'].astype(str).str.lower().apply(lambda x: 1 if 'unknown' in x or 'nan' in x else 0)
  df['IsDomeOpen']=df['StadiumType'].astype(str).str.lower().apply(lambda x: 1 if ('roof' in x or 'domed' in x or 'retr' in x or 'retr.' in x or 'dome' in x or 'bowl' in x) and ('roof-open' in x or 'open' in x) else 0)
  df['IsDomeOpen']=df['StadiumType'].astype(str).str.lower().apply(lambda x: 1 if 'retractable' in x else 0)
  df['IsDomeClosed']=df['StadiumType'].astype(str).str.lower().apply(lambda x: 1 if ('roof' in x or 'domed' in x or 'retr' in x or 'retr.' in x or 'dome' in x or 'bowl' in x) and ('closed' in x or 'roof-closed' in x) else 0)
  df['IsDomeClosed']=df['StadiumType'].astype(str).str.lower().apply(lambda x: 1 if 'domed' in x or 'bowl' in x else 0)
  df['IsIndoor']=df['StadiumType'].astype(str).str.lower().apply(lambda x: 1 if 'indoors' in x or 'indoor' in x else 0)
  df['IsIndoor']=np.where((df['IsDomeClosed']==1) | (df['IsDomeOpen']==1),1,df['IsIndoor'])
  df['IsOutdoor']=df['StadiumType'].astype(str).str.lower().apply(lambda x: 1 if 'heinz field' in x or 'outdor' in x or 'cloudy' in x or 'outside' in x or 'ourdoor' in x or 'oudoor' in x or 'outdoors' in x or 'outdoor' in x or 'open' in x or 'outddors' in x or 'outdoors' in x or 'oudoor' in x else 0)
  return df


In [ ]:
def preprooriginaldata(df):
    
    indoor=['Indoors','Indoor',]
    outdoor=['Heinz Field','Outdor', 'Cloudy','Outside','Ourdoor','Oudoor', 'Outdoors','Outdoor','Open','Outddors','Outdoors','Oudoor']
    dome_open=[ 'Retr. Roof - Open','Domed, Open', 'Domed, open','Retr. Roof-Open',  'Indoor, Open Roof','Outdoor Retr Roof-Open','Retractable Roof','Dome','Indoor, Open Roof','Retr. Roof - Open',]
    dome_closed=['Retr. Roof Closed','Dome, closed','Indoor, Roof Closed', 'Retr. Roof - Closed', 'Bowl','Domed','Closed Dome','Domed, closed','Retr. Roof-Closed','Closed Dome','Indoor, Roof Closed','Domed, closed','Retr. Roof - Closed']
    unknown=[np.nan,'nan',-999,'-999']
    df['StadiumType0']=df['StadiumType'].astype(str).replace(unknown,'unknown').replace(indoor,'indoor').replace(outdoor,'outdoor').replace(dome_open,'dome_open').replace(dome_closed,'dome_closed')
    
    cloudy=['Clear to Partly Cloudy','Coudy','cloudy','Mostly sunny','Cloudy, chance of rain','Mostly Coudy','Overcast', 'Cloudy, 50% change of rain','Hazy', 'Partly Clouidy','Party Cloudy','Partly cloudy','Partly Cloudy', 'Mostly cloudy', 'Cloudy and cold','Cloudy and Cool','Cloudy','Cloudy, fog started developing in 2nd quarter','Mostly Cloudy','Mostly cloudy','Cloudy and Cool','Cloudy, 50% change of rain','Cloudy','Partly Cloudy','Sun & clouds','Coudy']
    clear=['Clear and sunny','Clear and Sunny','Sunny, Windy', 'Mostly Sunny Skies','Sunny, highs to upper 80s', 'Sun & clouds', 'Heat Index 95','Partly clear','Fair','Sunny Skies','Clear skies','Sunny and clear','N/A Indoor','Partly Sunny','N/A (Indoors)', 'Mostly Sunny', 'Indoors', 'Clear Skies','Partly sunny', 'Clear and Cool',
       'Clear and cold', 'Sunny and cold', 'Indoor','Controlled Climate','Sunny and warm','Sunny', 'Clear','Clear and warm','Clear skies','Clear Skies','Mostly Sunny','Fair','Clear and warm','Sunny','Indoor', 'Clear','Controlled Climate','Cold','Indoors','Mostly sunny']
    rain=['Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.','Rain shower', 'Rainy','30% Chance of Rain','Cloudy, Rain','10% Chance of Rain','Light Rain','Rain likely, temps in low 40s.','Scattered Showers','Showers','Rain Chance 40%','Rain','Rain shower','Light Rain','Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.','Rain']
    snow=['Heavy lake effect snow','Snow', 'Cloudy, light snow accumulating 1-3"',]
    df['Weather0']=df['Weather'].astype(str).replace(unknown,'unknown').replace(snow,'snow').replace(cloudy,'cloudy').replace(clear,'clear').replace(rain,'rain')
    df['PlayerDay0']=df['PlayerDay']
    df['PlayerDay0']=np.where((df['PlayerDay0']>=365),df['PlayerDay0']-365,df['PlayerDay0'])
    df['PlayerDay0']=np.where(((df['PlayerDay0']>116) & (df['PlayerDay0']<365)),df['PlayerDay0']-365,df['PlayerDay0'])
    df['Preseason']=np.where((df['PlayerDay']<0) | ((df['PlayerDay']>116) & (df['PlayerDay']<365)),1,0)

    bins = np.linspace(0, 100, 11)
    groups = df.ix[df.Temperature>0,:].groupby(pd.cut(df.ix[df.Temperature>0,:].Temperature, bins))

    temp_df=pd.DataFrame(groups.mean().Temperature)
    df['Temperature0']=0
    for temp in range(len(groups.mean().Temperature)):
        df['Temperature0']=np.where(((df['Temperature']>bins[temp]) & (df['Temperature']<=bins[temp+1])),temp_df.iloc[temp,0],df['Temperature0'])
    #df=missingn(df)
    return df

In [ ]:
def inj(df): 
    df['RecoveryTime']=0
    df['RecoveryTime']=np.where(df['DM_M1']==1,1,df['RecoveryTime'])
    df['RecoveryTime']=np.where(df['DM_M7']==1,7,df['RecoveryTime'])
    df['RecoveryTime']=np.where(df['DM_M28']==1,28,df['RecoveryTime'])
    df['RecoveryTime']=np.where(df['DM_M42']==1,42,df['RecoveryTime'])
    #df=missingn(df)
    return df

Below are the representations of speed, acceleration, directional change and distance.
The analysis does not discredit player movements with contributing to injuries, however it proposes that the information given is inadequate as we cannot confirm at what time point in the duration of the play that the injury occurred. Therefore, our representation of speed, directional changes, acceleration and distance produced no predictive qualities. 

In [ ]:
%%time

radian_angle = (90 - players['dir']) * np.pi / 180.0
players['v_horizontal'] = np.abs(players['s'] * np.cos(radian_angle))
players['v_vertical'] = np.abs(players['s'] * np.sin(radian_angle))

temp_data= np.abs(players['time'].shift(-1)-players['time'])
temp_data=np.where( (temp_data>(temp_data.mean()+np.std(temp_data)*3)) ,temp_data.mean()+np.std(temp_data)*3,temp_data)
temp_data=np.where( (temp_data<(temp_data.mean()-np.std(temp_data)*3)) ,temp_data.mean()-np.std(temp_data)*3,temp_data)

players['time_vdiff']=temp_data
  

for cols in ['x', 'y','dis','dir','o','s']:
  temp_data= np.abs(players[cols].shift(-1)-players[cols])
  temp_data=np.where( (temp_data>(temp_data.mean()+np.std(temp_data)*3)) ,temp_data.mean()+np.std(temp_data)*3,temp_data)
  temp_data=np.where( (temp_data<(temp_data.mean()-np.std(temp_data)*3)) ,temp_data.mean()-np.std(temp_data)*3,temp_data)

  players['temp']=temp_data
    
  temp_data1= np.abs(players['temp'].shift(-1)-players['temp'])

  temp_data1=np.where( (temp_data1>(temp_data1.mean()+np.std(temp_data1)*3)) ,temp_data1.mean()+np.std(temp_data1)*3,temp_data1)
  temp_data1=np.where( (temp_data1<(temp_data1.mean()-np.std(temp_data1)*3)) ,temp_data1.mean()-np.std(temp_data1)*3,temp_data1)
 
  players['temp1']=temp_data1
  players[cols+'_vdiff_f1delta'] = np.where(players['time_vdiff']>0,players['temp']/players['time_vdiff'],0)
  players[cols+'_vdiff_adiff_f2delta'] = np.where(players['time_vdiff']>0,players['temp1']/players['time_vdiff'],0)
  
  ####Memory Management 
  #players=players.drop([cols],axis=1)
  players=players.drop(['temp','temp1'],axis=1)
print(players['time_vdiff'].min())
#players=players.drop(['time_vdiff'],axis=1)
%time players=reduce_mem_usage(players)
del temp_data,temp_data1
gc.collect()

In [ ]:
#Notes:
#Is data time series data in that players become more susceptible to injury with time?

In [ ]:
#250 players including/excluding injuries
players[:590]

In [ ]:
%time players=reduce_mem_usage(players)

In [ ]:
players

In [ ]:
len(players.PlayKey.unique())

In [ ]:
PlayKey_ls=players[idx].PlayKey
players=players[players.PlayKey.isin(PlayKey_ls)]
len(PlayKey_ls)

Below are the representations of play statistics for speed, acceleration, directional change and distance.

In [ ]:
%%time 
##Adding standard deviations for change in values/Quantify max orientation/direction change by time

def createstat(df):
  #df.drop_duplicates(['PlayKey'], keep='last',inplace=True)
  df1=df[idx]
  ls_cols=[x for x in df1.columns if x not in ['PlayKey', 'time', 'event']] 
  for cols in ls_cols:
    player_std = df[['PlayKey',cols]].groupby(['PlayKey'])\
                                .agg({cols:['max','std','min','mean']})\
                                         .reset_index()
    player_std.columns = ['PlayKey',cols+'_max',cols+'_std',cols+'_min',cols+'_mean']

    df1[cols+'_std']=player_std[cols+'_std']
    df1[cols+'_max']=player_std[cols+'_max']
    #df1[cols+'_min']=player_std[cols+'_min']
    #df1[cols+'_mean']=player_std[cols+'_mean']
    del player_std
    #df = pd.merge(df,player_std,on=['PlayKey'],how='inner')
  return df1

In [ ]:
#%time players0=createstat(players)
players0=players[idx]

In [ ]:
players0.shape

In [ ]:
plays=pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/PlayList.csv')
injuries=pd.read_csv('/kaggle/input/nfl-playing-surface-analytics/InjuryRecord.csv')
orig_colsinjuries=injuries.columns
orig_colsplays=plays.columns

In [ ]:
#%time injuries=reduce_mem_usage(injuries)
#%time plays=reduce_mem_usage(plays)
plays.isna().sum()

In [ ]:
injuries.isna().sum()

In [ ]:
%time plays=mapweatherandstadium(plays)

In [ ]:
%%time
injuries=inj(injuries)

In [ ]:
%%time
plays=preprooriginaldata(plays)

In [ ]:
len(plays.PlayKey.unique())

In [ ]:
double_injury_keys=list(injuries.PlayerKey.value_counts()[injuries.PlayerKey.value_counts()>1].index)
double_injury_keys

PlayerKey 33337 injured his foot in an earlier game 2 and reinjured it in game 8. PlayerKey 43540 injured his ankle in game 3 and reinjured it in game 7. PlayerKey 45950 injured his toes in game 6 and went on to injure his ankle in game 8. PlayerKey 44449 injured his knee in game 6 and went on to injure his ankle in game 28. 

Given the small dataset, we cannot conclude outside of PlayerKey 43540 and 33337, that one type of injury is casual to the next injury or establish a relationship. 

In [ ]:
injuries.loc[injuries.PlayerKey.isin([44449, 43540, 45950, 33337, 47307])].sort_values(by=['PlayerKey'])

In [ ]:
injuries.loc[injuries.PlayerKey.isin([44449, 43540, 45950, 33337, 47307])].sort_values(by=['PlayerKey']).to_csv('/kaggle/working/Doubleinjuries.csv')

DATA AUGMENTATION 

The idea was to augment the small injury dataset by duplicating knee and ankle injuries data and reversing the injury label. This was based off Player with PlayerKey 47307 who managed to injure his knee and ankle on the same play in the same game. Since the conditions that led to the double injury were similar, the assumption was that conditions affecting knee and ankle injuries are common. 

However, running a binary classification on the augmented dataset produced abnormally high log-loss scores i.e. 0.35 while non-augmented data achieved log-loss scores of 0.05. This proves that these two injuries had mutually exclusive sets of features/conditions affecting their occurrence. 

Therefore, any analysis should consider all the injuries individually. 

In [ ]:
#Data Augmentation
def daugment(injuries):
  inj1=injuries.loc[injuries.BodyPart=='Knee'].copy()
  inj2=injuries.loc[injuries.BodyPart=='Ankle'].copy()
  inj1['BodyPart']='Ankle'
  inj2['BodyPart']="Knee"
  inj3=pd.concat([inj1,inj2],axis=0)
  inj4=pd.concat([injuries,inj3],axis=0)
  return inj4

In [ ]:
#105 players with injuries
#%time injuries=daugment(injuries)

In [ ]:
#Plays including/excluding injuries for the 250 players
injuries.shape

In [ ]:
#250 players involved in 266960 plays
len(set(plays.PlayKey).intersection(set(players.PlayKey)))

In [ ]:
#No of unique playkeys in players 266960
len(players.PlayKey.unique())

In [ ]:
#No of unique playkeys in plays 267005
len(plays.PlayKey.unique())

In [ ]:
#Filtering plays for playkeys common to players
plays0=plays[plays.PlayKey.isin(players.PlayKey.unique())]


In [ ]:
#Merging plays0 and players0
playersplay = pd.merge(players0,plays0,how='outer', on='PlayKey')


In [ ]:
playersplay.shape

In [ ]:
playersplay['injuryplayerkey']=np.where(playersplay.PlayerKey.isin(injuries.PlayerKey),1,0)

In [ ]:
playersplay['injuryplaykey']=np.where(playersplay.PlayKey.isin(injuries.PlayKey),1,0)

In [ ]:
playersplay['injury']=np.where((playersplay['injuryplaykey']==1) & (playersplay['injuryplayerkey']==1),1,0)

In [ ]:
playersplay.PlayerKey.value_counts()

In [ ]:
injuries

The analysis concentrates on game conditions instead of play conditions, as a quarter of the injury data has no identifying play key. The choice to concetrate on game conditions shows better predictive quality across different models than play conditions. 

In [ ]:
playersplay=pd.merge(playersplay, injuries[['PlayerKey','GameID','BodyPart', 'Surface', 'DM_M1',
       'DM_M7', 'DM_M28', 'DM_M42', 'RecoveryTime']], how='outer', on=['PlayerKey','GameID'])

In [ ]:
playersplay=playersplay.drop(['injuryplaykey','injuryplayerkey'],axis=1)

In [ ]:
playersplay.columns

In [ ]:
playersplay['injuryKnee']=np.where((playersplay.BodyPart=='Knee'),1,0)
playersplay['injuryAnkle']=np.where((playersplay.BodyPart=='Ankle'),1,0)
playersplay['injuryToes']=np.where((playersplay.BodyPart=='Toes'),1,0)
playersplay['injuryFoot']=np.where((playersplay.BodyPart=='Foot'),1,0)
playersplay['injuryHeel']=np.where((playersplay.BodyPart=='Heel'),1,0)


In [ ]:
injuries.PlayKey.value_counts()

In [ ]:
set(playersplay.ix[(playersplay['injuryKnee']==1),:].PlayKey).intersection(set(playersplay.ix[(playersplay['injuryAnkle']==1),:].PlayKey))

In [ ]:
####Games with multiple injuries 
playersplay[playersplay.PlayKey.isin(list(set(playersplay.ix[(playersplay['injuryKnee']==1),:].PlayKey).intersection(set(playersplay.ix[(playersplay['injuryAnkle']==1),:].PlayKey))))]

In [ ]:
playersplay=reduce_mem_usage(playersplay)

The features considered to be noise below includes columns from the injury data as they produce data leak, and duplicate columns reproduced fro feature engineering.

In [ ]:
noise_ls=['PlayKey','PlayerKey', 'time', 'event', 'GameID','PlayKey','PlayerDay','DM_M1',
       'DM_M7', 'DM_M28', 'DM_M42', 'RecoveryTime','BodyPart', 'injury','Temperature','time_vdiff', 'StadiumType0', 'Weather0','Surface' ]
bodyp_exclude=[]
for bparts in [x for x in ['Knee', 'Foot', 'Ankle', 'Toes', 'Heel'] if x not in [choice]]:
  bodyp_exclude.append('injury'+bparts)
noise_ls+=bodyp_exclude

col_to_check='injury'+choice
baseline_ls0=list(orig_cols)+list(orig_colsinjuries)+list(orig_colsplays)
baseline_ls0.append('injury'+choice)
baseline_ls=[x for x in baseline_ls0 if x not in noise_ls]
baseline_ls


In [ ]:
playersplay.columns

In [ ]:
%%time
ls_cat=[x for x in playersplay.columns if x in playersplay.ix[:,playersplay.dtypes=='object'].columns] 
playersplay.replace([np.inf, -np.inf], np.nan)
[playersplay[c].replace(np.nan,'-999') for c in playersplay.columns[playersplay.isnull().any()] if c in ls_cat]
[playersplay[c].fillna('-999',inplace=True) for c in playersplay.columns[playersplay.isnull().any()] if c in ls_cat]

[playersplay[c].replace(np.nan,-999) for c in playersplay.columns[playersplay.isnull().any()] if c not in ls_cat]
[playersplay[c].fillna(-999, inplace=True) for c in playersplay.columns[playersplay.isnull().any()] if c not in ls_cat]


In [ ]:
col_mask=playersplay.isnull().any(axis=0) 
col_mask

In [ ]:
playersplay1=playersplay.drop(noise_ls,axis=1)
#playersplay1=playersplay[baseline_ls]

In [ ]:
playersplay1.isna().sum()

In [ ]:
#################################################################################################################################
#PLOTTING

In [ ]:
pip install dexplot

In [ ]:
import dexplot as dxp

In [ ]:
playersplay.columns

In [ ]:
dxp.jointplot('PlayerDay0','x', data=playersplay, hue='Surface')

In [ ]:
dxp.jointplot('PlayerDay0','y', data=playersplay, hue='Surface')

In [ ]:
dxp.jointplot('PlayerDay0','o', data=playersplay.loc[playersplay['o']>0], hue='Surface')

In [ ]:
dxp.jointplot('PlayerDay0','dir', data=playersplay.loc[playersplay['dir']>0], hue='Surface')

In [ ]:
dxp.jointplot('PlayerDay0','dis', data=playersplay.loc[playersplay['dis']>0], hue='Surface')

In [ ]:
dxp.jointplot('PlayerDay0','s', data=playersplay.loc[playersplay['s']>0], hue='Surface')

In [ ]:
dxp.jointplot("PlayerDay0",'v_horizontal', data=playersplay.loc[playersplay['v_horizontal']>0], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'v_vertical', data=playersplay.loc[playersplay['v_vertical']>0], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'x_vdiff_f1delta', data=playersplay.loc[(playersplay['x_vdiff_f1delta']>0) & (playersplay['x_vdiff_f1delta']<100)], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'x_vdiff_adiff_f2delta', data=playersplay.loc[(playersplay['x_vdiff_adiff_f2delta']>0) & (playersplay['x_vdiff_adiff_f2delta']<100)], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'y_vdiff_f1delta', data=playersplay.loc[(playersplay['y_vdiff_f1delta']>0) & (playersplay['y_vdiff_f1delta']<100)], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'y_vdiff_adiff_f2delta', data=playersplay.loc[(playersplay['y_vdiff_adiff_f2delta']>0) & (playersplay['y_vdiff_adiff_f2delta']<100)], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'o_vdiff_f1delta', data=playersplay.loc[(playersplay['o_vdiff_f1delta']>0) & (playersplay['o_vdiff_f1delta']<100)], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'o_vdiff_adiff_f2delta', data=playersplay.loc[(playersplay['o_vdiff_adiff_f2delta']>0) & (playersplay['o_vdiff_adiff_f2delta']<100)], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'dir_vdiff_f1delta', data=playersplay.loc[(playersplay['dir_vdiff_f1delta']>0) & (playersplay['dir_vdiff_f1delta']<100)], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'dir_vdiff_adiff_f2delta', data=playersplay.loc[(playersplay['dir_vdiff_adiff_f2delta']>0) & (playersplay['dir_vdiff_adiff_f2delta']<100)], hue="Surface")

In [ ]:
dxp.jointplot("PlayerDay0",'Temperature', data=playersplay, hue="Surface")

In the figure, we see that synthetic turf injuries outnumber natural turf injuries in ankles and toes and the reverse is true in feet. Heel injuries only occurred on natural turf and knee injuries were equivalent. 

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuries, hue='Surface',normalize='BodyPart')

Across all injuries, knees made the majority followed by ankles.

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuries, hue='Surface',normalize='all')

When normalized by surface type, knee injuries are the majority on natural turf while ankles are the majority on synthetic turf. 

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuries, hue='Surface',normalize='Surface')

In [ ]:
injuriesplays=pd.merge(injuries, plays, how='outer', on=['GameID', 'PlayKey', 'PlayerKey'])

In [ ]:
injuriesplays0=injuriesplays.ix[((injuriesplays.Surface=='Synthetic') | (injuriesplays.Surface=='Natural') ),:]

 Outdoor stadiums are heavily represented in the injury data for knees, feet and ankles.

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='StadiumType0',normalize='BodyPart')

Across all injuries, outdoor stadiums are heavily represented. 

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='StadiumType0',normalize='all')

When normalized by stadiumtype, knee injuries have a higher prevalence in indoor stadiums and outdoor stadiums while ankle injuries in open dome stadiums. 

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='StadiumType0',normalize='StadiumType0')

Knees and ankles afflict linebackers and wide receivers more while feet injuries afflict offensive linemen.

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='RosterPosition',normalize='BodyPart')

Across all injuries, linebackers and wide receivers were the groups most highly afflicted.

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='RosterPosition',normalize='all')

When normalized by number of players in each group, knee injuries afflict running backs more, feet injuries afflict tight ends more and ankle injuries afflict cornerbacks.

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='RosterPosition',normalize='RosterPosition')

Pass plays across ankle, feet and knees were the source of majority of the injuries. 

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='PlayType',normalize='BodyPart')

Across all injuries, pass plays were the majority source of injuries. 

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='PlayType',normalize='all')

When normalized by playtype, we see a higher prevalence of knee injuries for kickoffs returned/not returned, punts not returned for feet and punt returned for ankles. 

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='PlayType',normalize='PlayType')

Clear weather is the dominant weather in ankle and knee injuries while cloudy is dominant in feet injuries.

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='Weather0',normalize='BodyPart')

Across all injuries, clear weather is dominant except for feet.

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='Weather0',normalize='all')

When normalized by weather, we see that rainy weather has a higher prevalence for knee injuries and clear weather for ankle injuries. 

In [ ]:
dxp.aggplot(agg='BodyPart', data=injuriesplays0, hue='Weather0',normalize='Weather0')

In [ ]:
injplayplayers=pd.merge(injuriesplays0,players0, how='outer',on='PlayKey')

Credit: https://www.kaggle.com/cabonfim10/eda-nfl-1st-and-future

In [ ]:
Inj_Surf_BodyPart = injuries.groupby(['Surface','BodyPart'])['DM_M1','DM_M7','DM_M28','DM_M42'].sum().reset_index()
Inj_Surf_BodyPart

Outside of foot injuries, injuries sustained on synthetic turf (brown) either equal or exceed those on natural turf by count. Moving across the different groups to the more severe recovery times, synthetic turf injuries outnumber natural turf injuries. 

In [ ]:
Inj_Surf_BodyPart_New = pd.melt(Inj_Surf_BodyPart, id_vars=['Surface','BodyPart'], value_vars=['DM_M1','DM_M7','DM_M28','DM_M42'])

sns.catplot(x='BodyPart', y='value', hue='Surface', col='variable',
            data=Inj_Surf_BodyPart_New, kind='bar', height=6, aspect=.7)

Knee injuries taper off by day 75 in the season.

In [ ]:
if (choice=='Knee') or (choice=='Foot') or (choice=='Ankle'):
    #dxp.jointplot('experience', 'salary', data=emp, hue='gender')
 dxp.jointplot(x='PlayerDay0', y='BodyPart',hue='Weather0', data=injplayplayers.loc[injplayplayers.Preseason==0,:].drop_duplicates(subset=['PlayKey']))

In [ ]:
#del plays, injuries,

In [ ]:
##################################################################################################################################
#CATBOOST MODEL

In [ ]:
ls_cat=[x for x in playersplay1.drop(col_to_check,axis=1).columns if x in playersplay1.drop(col_to_check,axis=1).ix[:,playersplay1.dtypes==object]]
cat_featuresls1=[]
for i in ls_cat:
    cat_featuresls1.append(playersplay1.drop(col_to_check,axis=1).columns.get_loc(i))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(playersplay1.drop(col_to_check,axis=1),playersplay1[col_to_check], test_size=0.20, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split( X_test,y_test, test_size=0.50, random_state=42)

In [ ]:
pip install catboost

In [ ]:
#Scale positive weight
w=len(playersplay1.ix[playersplay1[col_to_check]==0,:])/len(playersplay1.ix[playersplay1[col_to_check]==1,:])
w

In [ ]:
ls_cols=[x for x in playersplay1.columns if x not in ['PlayKey', 'time', 'event']]

In [ ]:
#checking for collinearity
import seaborn as sns
import matplotlib.pyplot as plt

X=playersplay1[ls_cols]
sns.set(rc={'figure.figsize':(30, 30)})
corr = X.corr()
plt.figure() 
ax = sns.heatmap(corr, linewidths=.5, annot=True, cmap="YlGnBu", fmt='.1g')
plt.savefig('corr_heatmap.png')
plt.show()

In [ ]:
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.99:
            if columns[j]:
                columns[j] = False

feature_columns = X[corr.columns].columns[columns].values
drop_columns = X[corr.columns].columns[columns == False].values
print(feature_columns)
print('-'*73)
print(drop_columns)

In [ ]:
arr_corr=playersplay1[ls_cols].corr()
arr_corr

In [ ]:
new_fea=[]
new_feas=[]
num_corr=.85
for i in range(len(arr_corr.columns)):
    for k in range(i+1, len(arr_corr.columns)):
        val = arr_corr.iloc[k, i]
        col = arr_corr.columns[i]
        row = arr_corr.index[k]
        if val >= num_corr:
            print(col, "|", row, "|", round(val, 2))
            new_fea.append(col)
        elif val<=-num_corr:
            print(col, "|", row, "|", round(val, 2))
            new_feas.append(row)
new_fea=list(set(new_fea))
new_feas=list(set(new_feas))
new_col=[x for x in ls_cols if x not in new_fea and x not in new_feas]

In [ ]:
#CatBoost Model
from catboost import CatBoostClassifier, Pool, cv


simple_model1 = CatBoostClassifier(
    loss_function= 'Logloss',
    task_type= 'CPU',
    early_stopping_rounds=100,
    random_state= 47,
    use_best_model=False,
    eval_metric='Logloss',
    max_depth=9,
    thread_count=-1,
    verbose=50,
    #metric_period=50,
    l2_leaf_reg=9,
    scale_pos_weight=w,
    )
train_data1 = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_featuresls1,
                  thread_count=-1)
test_data1 = Pool(data=X_train1,
                  label=y_train1,
                  cat_features=cat_featuresls1,
                  thread_count=-1)

%time simple_model1.fit(train_data1,eval_set=test_data1,plot=True)
gc.collect()

In [ ]:
test_data2 = Pool(data=X_test1,
                  #label=y_test1,
                  cat_features=cat_featuresls1,
                  thread_count=-1)
y_pred=simple_model1.predict(test_data2, 
        prediction_type='Class', 
        ntree_start=0, 
        ntree_end=0, 
        thread_count=-1,
        verbose=None)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
catboost_score=precision_recall_fscore_support(y_test1, y_pred, average='macro')
catboost_score

In [ ]:
##Confusion Matrix
confusion_matrix(y_test1, y_pred)

In [ ]:
ls0=simple_model1.get_feature_importance(data=None,
                       fstr_type='Interaction',
                       prettified=False,
                       thread_count=-1,
                       verbose=False)
ls0

In [ ]:
ls=simple_model1.get_feature_importance(data=None,
                       #type=EFstrType.FeatureImportance,
                       prettified=False,
                       thread_count=-1,
                       verbose=False)
ls

In [ ]:
df_imp=pd.DataFrame(playersplay1.drop(col_to_check,axis=1).columns, columns=["feature"])
df_imp['cbimportance']=ls


In [ ]:
df_imp=df_imp.sort_values(by=['cbimportance'],ascending=False)
df_imp['cbnumber']=range(1,len(df_imp)+1)
df_imp

In [ ]:
ls_to_nn=list(df_imp.ix[df_imp.cbnumber<=20,:].feature)
ls_to_nn_exclude=list(df_imp.feature)
ls_to_nn_exclude

In [ ]:
df_interact=pd.DataFrame(ls0, columns=['feature1','feature2','relativeimportance'])
df_interact

In [ ]:
ls_cols=X_train.columns

In [ ]:
df_interact['f1name']=0
df_interact['f2name']=0
for row in range(len(df_interact)):
    df_interact['f1name'][row]=ls_cols[df_interact['feature1'][row]]
    df_interact['f2name'][row]=ls_cols[df_interact['feature2'][row]]

In [ ]:
df_interact.to_csv('/kaggle/working/FeatureInteraction.csv')
df_interact

In [ ]:
for rows in range(len(df_interact)):
    print(df_interact.f1name[rows])
    print(df_interact.f2name[rows])
    print('\n')

In [ ]:
#######################################################################################################################
#LIGHTGBM MODEL

In [ ]:
del X_train, X_test, y_train, y_test,X_train1, X_test1, y_train1, y_test1


In [ ]:
X_t=playersplay.drop(noise_ls,axis=1).drop(col_to_check,axis=1)
y_t=playersplay[col_to_check]

ls_cat0=[x for x in playersplay.drop(noise_ls,axis=1).drop(col_to_check,axis=1).columns if x in playersplay.drop(noise_ls,axis=1).drop(col_to_check,axis=1).ix[:,playersplay.dtypes==object]]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for c in ls_cat0: 
    le.fit(list(X_t[c].values))
    X_t[c] = le.transform(list(X_t[c].values))

In [ ]:
from sklearn.model_selection import train_test_split
X_train0, X_test0, y_train0, y_test0 = train_test_split(X_t,y_t, test_size=0.20, random_state=42)
X_train00, X_test00, y_train00, y_test00 = train_test_split(X_test0,y_test0, test_size=0.50, random_state=42)

In [ ]:
len(X_test00)

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train0, y_train0,)
test_data = lgb.Dataset(X_train00, y_train00)
#train_data0 = lgb.Dataset(X_t, y_t,)

In [ ]:
###Light GBM Comparison/Implements Elastic, both L1 and L2
import lightgbm as lgb
params={
    'max_bin':63,
    'max_depth':-1,
    'num_leaves':70,
    'num_iterations':500,
    'min_child_weight':0.03,
    'feature_fraction ':0.4,
    'bagging_fraction':0.4,
    'min_data_in_leaf': 1,
    'objective': 'binary',
    'learning_rate':0.01,
    'boosting_type': "gbdt",
    'bagging_seed': 11,
    'metric':'binary_logloss',
    'random_state':47,
    'num_thread ': -1,
    'scale_pos_weight':w,
    'lambda_l1':7,
    'lambda_l2':7,
}
gbmrfe = lgb.LGBMClassifier(
    max_bin =63,
    max_depth=-1,
    num_leaves = 70,
    num_iterations = 1000,
    min_child_weight= 0.03,
    feature_fraction = 0.4,
    bagging_fraction= 0.4,
    min_data_in_leaf= 1,
    objective= 'binary',
    learning_rate= 0.05,
    boosting_type= "gbdt",
    bagging_seed= 11,
    metric= 'binary_logloss',
    random_state= 47,
    num_thread = -1,
    scale_pos_weight=w,
    lambda_l1=7,
    lambda_l2=7,
    importance_type='gain',
    #categorical_feature=ls_cat0,

)

%time gbmrfe.fit(X_train0, y_train0,eval_set=(X_train00, y_train00))
gc.collect()
#%time gbmrfe=lgb.cv(params, train_data0, nfold=5, stratified=True, shuffle=True,verbose_eval=1)

In [ ]:
gbmrfe_df=pd.DataFrame(data=gbmrfe.feature_importances_,)
gbmrfe_df['feature']=X_t.columns
gbmrfe_df=gbmrfe_df.sort_values(by=0,ascending=False)
gbmrfe_df['lgbmimportance']=gbmrfe_df.ix[:,0]
gbmrfe_df['lgbmnumber']=range(1,len(gbmrfe_df)+1)
#gbmrfe_df.sort_values(by=0,ascending=False)
gbmrfe_df

In [ ]:
lgb.plot_importance(gbmrfe,importance_type ='gain')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
ypredlgbm=gbmrfe.predict(X_test00)
lgbm_score=precision_recall_fscore_support(y_test00, ypredlgbm, average='macro')
lgbm_score

In [ ]:
catboost_score,lgbm_score

In [ ]:
##Confusion Matrix
confusion_matrix(y_test00, ypredlgbm)

In [ ]:
#####################################################################################################################
#RANDOM FOREST MODEL

In [ ]:
np.where(X_t.values >= np.finfo(np.float32).max)

In [ ]:
np.isnan(X_t.values.any())

In [ ]:
np.finfo(np.float32)

In [ ]:
X_t[:] = np.nan_to_num(X_t)

In [ ]:
%%time

X_t.replace([np.inf, -np.inf], np.nan)
X_t.replace(np.nan,-999)
X_t.fillna(-999,inplace=True)
X_t=reduce_mem_usage(X_t)

In [ ]:
del X_train0, X_test0, y_train0, y_test0,X_train00, X_test00, y_train00, y_test00 


In [ ]:
from sklearn.model_selection import train_test_split
X_train0rf, X_test0rf, y_train0rf, y_test0rf = train_test_split(X_t,y_t, test_size=0.20, random_state=42)


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
clf = ExtraTreesClassifier(n_estimators=1000, random_state=47,n_jobs=-1,class_weight='balanced_subsample',verbose=1,oob_score=True,bootstrap=True,criterion='entropy' )
clf.fit(X_train0rf, y_train0rf)
gc.collect()

In [ ]:
y_predrf=clf.predict(X_test0rf)
rf_score=precision_recall_fscore_support(y_test0rf, y_predrf, average='macro')
catboost_score,lgbm_score,rf_score

In [ ]:
##Confusion Matrix
confusion_matrix(y_test0rf, y_predrf)

In [ ]:

rf_df=pd.DataFrame(data=clf.feature_importances_,)
rf_df['feature']=X_train0rf.columns
rf_df=rf_df.sort_values(by=0,ascending=False)
rf_df['rfimportance']=rf_df.ix[:,0]
rf_df['rfnumber']=range(1,len(rf_df)+1)
rf_df

In [ ]:
df_agg_importance=pd.merge(rf_df,gbmrfe_df, how='inner', on='feature')
df_agg_importance=pd.merge(df_agg_importance,df_imp, how='inner', on='feature')
df_agg_importance=df_agg_importance[['feature','cbnumber', 'rfnumber', 'lgbmnumber',
       ]]
round(df_agg_importance.sort_values(by='cbnumber',ascending=True),3).to_csv('/kaggle/working/FeatureImportance.csv')
round(df_agg_importance.sort_values(by='cbnumber',ascending=True),3)

In [ ]:
##############################################################################################################
#RECURSIVE FEATURE ELIMINATION WITH CATBOOST

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score,log_loss
import lightgbm as lgb 
import matplotlib.pyplot as plt



# Build a classification task using prior LGBM Model



gc.collect()


rfecv = RFECV(estimator=gbmrfe, step=1, cv=StratifiedKFold(2),min_features_to_select=1, verbose=5,scoring='neg_log_loss')

%time rfecv.fit(X_t,y_t)
rfecv.get_params
rfecv.support_
gc.collect()
print("Optimal number of features : %d" % rfecv.n_features_)
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()


In [ ]:
####Features to keep
X_t.ix[:,rfecv.support_].columns

In [ ]:
####Features Eliminated
X_t.ix[:,rfecv.support_==False].columns

In [ ]:
##########################################################################################################################
#FACTOR ANALYSIS

Factor Analysis can be used as a form of exploratory feature analysis by dimension reduction.  

Factor analysis was done to discover hidden relationships between features by extracting maximum common variance. After calculating eigen values, we create 13 factors which explain 0.6867 of the total variances. Even after calculating for 46 factors, the total explained variance was approximately 0.7055. 

In [ ]:
pip install factor_analyzer

In [ ]:
X_t=playersplay.drop(noise_ls,axis=1).drop(col_to_check,axis=1)

ls_cat0=[x for x in playersplay.drop(noise_ls,axis=1).drop(col_to_check,axis=1).columns if x in playersplay.drop(noise_ls,axis=1).drop(col_to_check,axis=1).ix[:,playersplay.dtypes==object]]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for c in ls_cat0: 
    le.fit(list(X_t[c].values))
    X_t[c] = le.transform(list(X_t[c].values))

In [ ]:
from factor_analyzer import FactorAnalyzer,Rotator

In [ ]:
%%time
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(X_t)

In [ ]:
chi_square_value, p_value

In [ ]:
%%time
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(X_t)

In [ ]:
kmo_model

In [ ]:
%%time
fa = FactorAnalyzer()
fa.fit(X_t)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()


In [ ]:
ev

In [ ]:
plt.scatter(range(1,X_t.shape[1]+1),ev)
plt.plot(range(1,X_t.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

In [ ]:
fa=FactorAnalyzer(bounds=(0.005, 1), impute='median', is_corr_matrix=False,
                method='minres', n_factors=14, rotation=None, rotation_kwargs={},
                use_smc=True)
fa.fit(X_t)
rotator = Rotator()
rot_fa=rotator.fit_transform(fa.loadings_)
rotate_df=pd.DataFrame(rot_fa)
rotate_df['Features']=X_t.columns
new_order = [14,0,1,2,3,4,5,6,7,8,9,10,11,12,13]
rotate_df[rotate_df.columns[new_order]]

In [ ]:
fa.get_factor_variance()[2]

In [ ]:
for cols in rotate_df.drop(['Features'],axis=1).columns:
  final_df = rotate_df[['Features',cols]].sort_values(by=[cols], ascending=False)
  print(cols)
  print(final_df.head(3))